## Hyperparam Tuning

Now that we know which models are performing better, it's time to perform cross validation and tune hyperparameters.
- Do a google search for hyperparameter ranges for each type of model.

GridSearch/RandomSearch are a great methods for checking off both of these tasks.

There is a fairly significant issue with this approach for this particular problem (described below). But in the interest of creating a basic functional pipeline, you can just use the default Sklearn methods for now.

## Preventing Data Leakage in Tuning - STRETCH

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it, if you have time!**

BUT we have a problem - if we calculated a numerical value to encode city (such as the mean of sale prices in that city) on the training data, we can't cross validate 
- The rows in each validation fold were part of the original calculation of the mean for that city - that means we're leaking information!
- While sklearn's built in functions are extremely useful, sometimes it is necessary to do things ourselves

You need to create two functions to replicate what Gridsearch does under the hood. This is a challenging, real world data problem! To help you out, we've created some psuedocode and docstrings to get you started. 

**`custom_cross_validation()`**
- Should take the training data, and divide it into multiple train/validation splits. 
- Look into `sklearn.model_selection.KFold` to accomplish this - the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) shows how to split a dataframe and loop through the indexes of your split data. 
- Within your function, you should compute the city means on the training folds just like you did in Notebook 1 - you may have to re-join the city column to do this - and then join these values to the validation fold

This psuedocode may help you fill in the function:

```python
kfold = KFold() # fit sklearn k folds on X_train
train_folds = []
val_folds = []
for training_index, val_index in kfold.split(X_train):
    train_fold, val_fold = #.iloc loop variables on X_train

    # recompute training city means like you did in notebook 1 
    # merge to validation fold
        
    train_folds.append(train_fold)
    val_folds.append(val_fold)

    return train_folds, val_folds
```


**`hyperparameter_search()`**
- Should take the validation and training splits from your previous function, along with your dictionary of hyperparameter values
- For each set of hyperparameter values, fit your chosen model on each set of training folds, and take the average of your chosen scoring metric. [itertools.product()](https://docs.python.org/3/library/itertools.html) will be helpful for looping through all combinations of hyperparameter values
- Your function should output the hyperparameter values corresponding the highest average score across all folds. Alternatively, it could also output a model object fit on the full training dataset with these parameters.


This psuedocode may help you fill in the function:

```python
hyperparams = # Generate hyperparam options with itertools
hyperparam-scores = []
for hyperparam-combo in hyperparams:

    scores = []

    for folds in allmyfolds:
        # score fold the fold with the model/ hyperparams
        scores.append(score-fold)
        
    score = scores.mean()
    hyperparam-scores.append(score)
# After loop, find max of hyperparam-scores. Best params are at same index in `hyperparams` loop iteratble
```

Docstrings have been provided below to get you started. Once you're done developing your functions, you should move them to `functions_variables.py` to keep your notebook clean 

Bear in mind that these instructions are just one way to tackle this problem - the inputs and output formats don't need to be exactly as specified here.

In [62]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, KFold, cross_val_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

from sklearn.metrics import mean_squared_error, r2_score, make_scorer

from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [71]:
X_train_scaled = pd.read_csv('../data/processed/X_train_scaled.csv')
X_test_scaled = pd.read_csv('../data/processed/X_test_scaled.csv')
y_train = pd.read_csv('../data/processed/y_train.csv')
y_test = pd.read_csv('../data/processed/y_test.csv')

In [3]:
# develop your custom functions here

def custom_cross_validation(training_data, n_splits =5):
    '''creates n_splits sets of training and validation folds

    Args:
      training_data: the dataframe of features and target to be divided into folds
      n_splits: the number of sets of folds to be created

    Returns:
      A tuple of lists, where the first index is a list of the training folds, 
      and the second the corresponding validation fold

    Example:
        >>> output = custom_cross_validation(train_df, n_splits = 10)
        >>> output[0][0] # The first training fold
        >>> output[1][0] # The first validation fold
        >>> output[0][1] # The second training fold
        >>> output[1][1] # The second validation fold... etc.
    '''

    kfold = KFold(n_splits=n_splits)
    training_folds = []
    validation_folds = []

    for train_index, val_index in kfold.split(train_df):
      train_fold = train_df.iloc[train_index].copy()
      val_fold = train_df.iloc[val_index].copy()

      # Compute city means in the training fold
      city_means = train_fold.groupby(city_column)[target_column].mean().reset_index()
      city_means.columns = [city_column, f'{target_column}_city_mean']

      # Merge these means into the validation fold based on the city column
      val_fold = val_fold.merge(city_means, on=city_column, how='left')

      training_folds.append(train_fold)
      validation_folds.append(val_fold)

    

    return training_folds, validation_folds

def hyperparameter_search(training_folds, validation_folds, param_grid):
    '''outputs the best combination of hyperparameter settings in the param grid, 
    given the training and validation folds

    Args:
      training_folds: the list of training fold dataframes
      validation_folds: the list of validation fold dataframes
      param_grid: the dictionary of possible hyperparameter values for the chosen model

    Returns:
      A list of the best hyperparameter settings based on the chosen metric

    Example:
        >>> param_grid = {
          'max_depth': [None, 10, 20, 30],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 2, 4],
          'max_features': ['sqrt', 'log2']} # for random forest
        >>> hyperparameter_search(output[0], output[1], param_grid = param_grid) 
        # assuming 'ouput' is the output of custom_cross_validation()
        [20, 5, 2, 'log2'] # hyperparams in order
    '''

    return hyperparameters


## Hyperparam Tuning

In [ ]:
# using GridsearchCV/ RandomsearchCV (MVP)
# Decision Tree
# Random Forest
# Gradient Booster
# SVR
# Gradient booster

In [104]:
# Random Forest model
forest_model = RandomForestRegressor(random_state=42)
forest_model.fit(X_train_scaled, y_train)
y_pred_forest = forest_model.predict(X_test_scaled)

mse_forest = mean_squared_error(y_test, y_pred_forest)
r2_forest = r2_score(y_test, y_pred_forest)

print(mse_forest, r2_forest)

/var/folders/_0/kyp2mnk90qn2fz6spd6zpns40000gn/T/ipykernel_4778/2840024053.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_model.fit(X_train_scaled, y_train)


48807475.34689576 0.9995212826123541


In [105]:
# RandomForestRegressor
param_grid = {
    'n_estimators': [50, 65 ,70], #To find a balance between computational cost and performance improvement
    'max_features': [1.0, 'sqrt', 'log2'], # Selected to find the best split
    'max_depth': [None, 5, 10], # Selected to provide range between shallow and deep trees. Shallow trees help to prevent overfitting
    'min_samples_split': [2, 4, 3], #min samples required to split an internal node
    'min_samples_leaf': [1, 2, 3], #selcted values to indicate minimum samples required to make a leaf node
    'bootstrap': [True, False] #true bootsrapping can help reduce overfitting 
}


rf_grid_search = GridSearchCV(estimator=forest_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)

rf_grid_search.fit(X_train_scaled, y_train)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector 

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_depth': [None, 5, 10],
                         'max_features': [1.0, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 4, 3],
                         'n_estimators': [50, 65, 70]},
             scoring='neg_mean_squared_error')

In [106]:
rf_best_params = rf_grid_search.best_params_ # to determine the parameters that showed the best result
rf_best_model = rf_grid_search.best_estimator_ # to extract the Random Forest best model

print(f"The parameters for the best Random Forest Regressor model includes {rf_best_params}")

The parameters for the best Forest Regressor includes {'bootstrap': False, 'max_depth': None, 'max_features': 1.0, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 70}


In [107]:
y_pred= rf_best_model.predict(X_test_scaled)

mse_rf_grid_search = mean_squared_error(y_test, y_pred)
r2_rf_grid_search = r2_score(y_test, y_pred)

print(mse_rf_grid_search, r2_rf_grid_search)

16323156.6687989 0.999839897907788


In [114]:
print(f"We noted that the best model for the Random Forest model produced a lower error for both the mean squared error and the R2 score \n Random Forest Best model parameter: R2 score of {r2_rf_grid_search:.4f} and Mean Squared Error score of {mse_rf_grid_search:.2f} \n Default parameter: R2 score of {r2_forest:.4f} and Mean Squared Error score of {mse_forest:.2f}.") 

We noted that the best model for the Random Forest model produced a lower error for both the mean squared error and the R2 score 
 Random Forest Best model parameter: R2 score of 0.9998 and Mean Squared Error score of 16323156.67 
 Default parameter: R2 score of 0.9995 and Mean Squared Error score of 48807475.35.


In [120]:
with open('../models/random_forest_model.pkl', 'wb') as file:
    pickle.dump(rf_grid_search, file)

In [78]:
xgb_model = XGBRegressor(random_state=42)
xgb_model.fit(X_train_scaled, y_train)

y_test_xgb = xgb_model.predict(X_test_scaled)

mse_xgb = mean_squared_error(y_test, y_test_xgb)
r2_xgb = r2_score(y_test, y_test_xgb)

XGB_metrics = {'MSE':mse_xgb, 'R2':r2_xgb}
XGB_metrics

{'MSE': 29186569.95928856, 'R2': 0.9997137299475961}

In [81]:
# XGbooster
param_grid = {
    'max_depth': [5, 10, 15, 20], 
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [50, 100, 70],
    'gamma': [0, 0.1, 0.3],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=1)




In [83]:
xgb_grid_search.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0], 'gamma': [0, 0.1, 0.3],
                         'learning_rate': [0.1, 0.01, 0.001],
                         'max_depth': [5, 10, 15, 20],
                         'n_estimators': [50, 100, 70],
                         'subsample': [0.8, 1.0]},
             scoring='neg_mean_squared_error', verbose=1)

In [122]:
xgb_best_params = xgb_grid_search.best_params_ # to determine the parameters that that have the XGbooster model
xgb_best_model = xgb_grid_search.best_estimator_ # to extract the XGbooster best model

print(f"The parameters for the best XGBooster model includes {xgb_best_params}")

The parameters for the best XGBooster model includes {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 20, 'n_estimators': 100, 'subsample': 0.8}


In [89]:
y_pred= xgb_best_model.predict(X_test_scaled) #to predict our y(sale-price) with our tuned XGBooster model

mse_xgb_grid_search = mean_squared_error(y_test, y_pred)
r2_xgb_grid_search = r2_score(y_test, y_pred)

print(mse_xgb_grid_search, r2_xgb_grid_search)

11038580.088273002 0.9998917305149341


In [121]:
print(f"We noted that the model with the tuned parameters performed better in the XGBooster tuned in comparison to the model with the XGBooster with default parameters \nXGBoosterBest model parameter: R2 score of {r2_xgb_grid_search:.4f} and Mean Squared Error score of {mse_xgb_grid_search:.2f}. \nDefault parameter: R2 score of {r2_xgb:.4f} and Mean Squared Error score of {mse_xgb:.2f}.")


We noted that the model with the tuned parameters performed better in the XGBooster tuned in comparison to the model with the XGBooster with default parameters 
XGBoosterBest model parameter: R2 score of 0.9999 and Mean Squared Error score of 11038580.09. 
Default parameter: R2 score of 0.9997 and Mean Squared Error score of 29186569.96.


In [119]:
#saving the xgbooster best model

with open('../models/XGBoost_model.pkl', 'wb') as file:
    pickle.dump(xgb_grid_search, file)

We want to make sure that we save our models.  In the old days, one just simply pickled (serialized) the model.  Now, however, certain model types have their own save format.  If the model is from sklearn, it can be pickled, if it's xgboost, for example, the newest format to save it in is JSON, but it can also be pickled.  It's a good idea to stay with the most current methods. 
- you may want to create a new `models/` subdirectory in your repo to stay organized

## Building a Pipeline (Stretch)

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its highly recommended you complete it if you have time!**

Once you've identified which model works the best, implement a prediction pipeline to make sure that you haven't leaked any data, and that the model could be easily deployed if desired.
- Your pipeline should load the data, process it, load your saved tuned model, and output a set of predictions
- Assume that the new data is in the same JSON format as your original data - you can use your original data to check that the pipeline works correctly
- Beware that a pipeline can only handle functions with fit and transform methods.
- Classes can be used to get around this, but now sklearn has a wrapper for user defined functions.
- You can develop your functions or classes in the notebook here, but once they are working, you should import them from `functions_variables.py` 

In [ ]:
# Build pipeline here

Pipelines come from sklearn.  When a pipeline is pickled, all of the information in the pipeline is stored with it.  For example, if we were deploying a model, and we had fit a scaler on the training data, we would want the same, already fitted scaling object to transform the new data with.  This is all stored when the pipeline is pickled.
- save your final pipeline in your `models/` folder

In [ ]:
# save your pipeline here